In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np

In [2]:
#image_folder = 'img/GEE-imgs'

paths = load_json(os.path.join('conf', 'paths.json'))
shp = load_json(os.path.join('conf', 'shp.json'))
conf = load_json(os.path.join('conf', 'conf.json'))
shp_path = paths['shp']

img_source = conf['img_source']
grid_save = os.path.join('shp', f"tiles.shp")


In [3]:
image_folder = os.path.join('img', 'opt')
labels_folder = os.path.join('img', 'labels')

image_p = os.path.join(image_folder, '2017', f'PA_S2_2017_B1_B2_B3_crop.tif')

v_def_until_2007 = ogr.Open(os.path.join(shp_path, 'accumulated_deforestation_area.shp'))
l_def_until_2007 = v_def_until_2007.GetLayer()

v_def_yearly = ogr.Open(os.path.join(shp_path, 'yearly_deforestation_area.shp'))
l_def_yearly = v_def_yearly.GetLayer()

data = gdal.Open(image_p, gdalconst.GA_ReadOnly)

geo_transform = data.GetGeoTransform()
#x_min = geo_transform[0]
#y_max = geo_transform[3]
#x_max = x_min + geo_transform[1] * data.RasterXSize
#y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize

crs = data.GetSpatialRef()
proj = data.GetProjection()

pixel_width = geo_transform[1]

## Map Gen

In [5]:
last_year = 2018
ref_year = 2006

output_2019 = os.path.join(labels_folder, f'previous_def_{last_year+1}.tif')

target_ds_2019 = gdal.GetDriverByName('GTiff').Create(output_2019, x_res, y_res, 1, gdal.GDT_Float32)
#target_ds.SetGeoTransform((x_min, geo_transform[1], 0, y_min, 0, geo_transform[5]))
target_ds_2019.SetGeoTransform(geo_transform)
target_ds_2019.SetSpatialRef(crs)
target_ds_2019.SetProjection(proj)
band_2019 = target_ds_2019.GetRasterBand(1)
#NoData_value = 0
#band.SetNoDataValue(NoData_value)
band_2019.FlushCache()


for year in range(last_year, 2007, -1):
    where = f'"year"={year}'
    l_def_yearly.SetAttributeFilter(where)
    v = (year - ref_year)/(last_year - ref_year)
    print(v)
    gdal.RasterizeLayer(target_ds_2019, [1], l_def_yearly, burn_values=[v])

year = 2007
v = (year - ref_year)/(last_year - ref_year)
print(v)
gdal.RasterizeLayer(target_ds_2019, [1], l_def_until_2007, burn_values=[v])

target_ds_2019 = None

1.0
0.9166666666666666
0.8333333333333334
0.75
0.6666666666666666
0.5833333333333334
0.5
0.4166666666666667
0.3333333333333333
0.25
0.16666666666666666
0.08333333333333333
